In [ ]:
!pip install transformers datasets

In [2]:
from datasets import load_dataset

In [3]:
import json
import pandas as pd

In [16]:
df = pd.read_csv(r'/content/1.0-all_data-api_usage-tanh_score.csv',  engine='python', index_col=0).iloc[:, :-7]

In [17]:
df.head()

,Q_CreationDate,Title,Question,Answer,Score,Is_accepted,N_answers,Q_Id
0,2008-08-05 22:26:00.797,Create a directly-executable cross-platform GU...,Python works on multiple platforms and can be ...,Since python is installed on nearly every non-...,0.186186,False,2,4
1,2008-08-05 22:26:00.797,Create a directly-executable cross-platform GU...,Python works on multiple platforms and can be ...,I'm not sure that this is the best way to do i...,0.124933,False,2,4
2,2008-08-22 10:51:19.057,"How do content discovery engines, like Zemanta...",I was wondering how as semantic service like O...,Open Calais probably use language parsing tech...,0.000000,False,1,17
3,2008-08-22 15:34:13.760,How to download a file over HTTP?,I have a small utility that I use to download ...,Following are the most commonly used calls for...,0.293455,False,1,18
4,2008-08-29 04:59:31.200,Find out how much memory is being used by an o...,How would you go about finding out how much me...,For big objects you may use a somewhat crude b...,0.161830,False,1,30


In [ ]:
all_data = []
all_data.extend(data['train'])
all_data.extend(data['val'])
all_data.extend(data['test'])

In [ ]:
df = pd.DataFrame.from_dict(all_data).iloc[:, :-7]

In [18]:
df['Q_Id'] = df.groupby('Q_CreationDate').ngroup()

In [60]:
import pandas as pd
import numpy as np

def binary_comparison(answers):
    """Returns tuples of answers, first always best"""
    pairs = []
    
    for i in range(len(answers)-1):
        for j in range(i+1, len(answers)):
            if answers[i]["Score"] > answers[j]["Score"]:
                pairs.append((answers[i]["Answer"], answers[j]["Answer"]))
            elif answers[i]["Score"] < answers[j]["Score"]:
                pairs.append((answers[j]["Answer"], answers[i]["Answer"]))
            else:
                # random choice when the scores are equal
                if np.random.choice([True, False]):
                    pairs.append((answers[i]["Answer"], answers[j]["Answer"]))
                else:
                    pairs.append((answers[j]["Answer"], answers[i]["Answer"]))
    return pairs
    
def preprocess(df):
    """Returns paired answers (j is better than k)."""

    MAX_PAIRS_PER_QUESTION = 10
    
    # initialize empty lists for new samples
    new_examples = {'q_id': [], 'title': [], "question": [], "response_j": [], "response_k": []}
    
    for qid in df['Q_Id'].unique():
        question_df = df[df['Q_Id'] == qid]
        if len(question_df) <= 1:
            continue
        
        # get pairs where first is always the better one
        pairs = binary_comparison(question_df.to_dict('records'))
        
        # sample if we get more pairs than maximum
        if len(pairs) > MAX_PAIRS_PER_QUESTION:
            indices = np.random.choice(list(range(len(pairs))), MAX_PAIRS_PER_QUESTION, replace=False)
            pairs = [pairs[i] for i in indices]
        
        # construct the samples
        for pair in pairs:
            new_examples["q_id"].append(question_df.iloc[0]['Q_Id'])
            new_examples["title"].append(question_df.iloc[0]['Title'])
            new_examples["question"].append(question_df.iloc[0]['Question'])
            new_examples["response_j"].append(pair[0])
            new_examples["response_k"].append(pair[1])
    
    return pd.DataFrame(new_examples)


In [61]:
contrast_df = preprocess(df)

In [67]:
contrast_df.to_csv('/content/python-api_usage-contrast-tanh_score.csv', index=False)

In [65]:
len(contrast_df.q_id.unique())

1272

In [ ]:
import json
import pandas as pd

In [36]:
train_df = df.iloc[:-3000]
val_df = df.iloc[-3000:-2000]
test_df = df.iloc[-2000:]

In [37]:
train_df.to_csv('/content/train.csv', index=False)
val_df.to_csv('/content/validation.csv', index=False)
test_df.to_csv('/content/test.csv', index=False)

In [45]:
data_files = {"train": '/content/train.csv',
              'validation': '/content/validation.csv',
              'test': "/content/test.csv"}
dataset = load_dataset("csv", data_files=data_files, split='test')

In [53]:
df.columns

Index(['Q_CreationDate', 'Title', 'Question', 'Answer', 'Score', 'Is_accepted',
       'N_answers', 'Q_Id'],
      dtype='object')

In [47]:
dataset.features

{'Q_CreationDate': Value(dtype='string', id=None),
 'Title': Value(dtype='string', id=None),
 'Question': Value(dtype='string', id=None),
 'Answer': Value(dtype='string', id=None),
 'Score': Value(dtype='float64', id=None),
 'Is_accepted': Value(dtype='bool', id=None),
 'N_answers': Value(dtype='int64', id=None),
 'Q_Id': Value(dtype='int64', id=None)}

In [48]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as d

In [50]:
!git lfs install

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.


In [51]:
!git clone https://huggingface.co/datasets/namespace/Myashka/SO-Python_QA-API_Usage-tanh_score

Cloning into 'SO-Python_QA-API_Usage-tanh_score'...
fatal: could not read Username for 'https://huggingface.co': No such device or address


In [52]:
!huggingface-cli repo create SO-Python_QA-API_Usage-tanh_score --type dataset

git version 2.25.1
git-lfs/2.9.2 (GitHub; linux amd64; go 1.13.5)

You are about to create datasets/Myashka/SO-Python_QA-API_Usage-tanh_score
Proceed? [Y/n] y
409 Client Error: Conflict for url: https://huggingface.co/api/repos/create (Request ID: Root=1-6470a0b8-633ff1f556bdc89c4e5b0144)

You already created this dataset repo
{"error":"You already created this dataset repo","url":"https://huggingface.co/datasets/Myashka/SO-Python_QA-API_Usage-tanh_score"}
